## 티빙클립 크롤링

### 프로젝트 생성

In [51]:
!scrapy startproject tiving

New Scrapy project 'tiving', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/notebook/크롤링프로젝트/tiving

You can start your first spider with:
    cd tiving
    scrapy genspider example example.com


### 크롤링 코드

In [2]:
import requests
import scrapy
import json

from scrapy.http import TextResponse

#### apiKey 가져오기

In [3]:
url = 'http://www.tving.com/pick/player/top/top'
req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding='utf-8')

In [4]:
apiKey = response.text.split("apiKey=")[1].split("&")[0]
apiKey

'1e7952d0917d6aab1f0293a063697610'

#### apiKey로 데이터 수집하기

In [5]:
page, page_size = 1, 100

url = 'http://api.tving.com/v2/media/pick/clips?\
screenCode=CSSD0100&networkCode=CSND0900&osCode=CSOD0900&teleCode=CSCD0900\
&apiKey={}&pageNo={}&pageSize={}&isuse=Y&order=viewDay&personal=Y'.format(apiKey, page, page_size)

req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding='utf-8')
response

<200 http://api.tving.com/v2/media/pick/clips?screenCode=CSSD0100&networkCode=CSND0900&osCode=CSOD0900&teleCode=CSCD0900&apiKey=1e7952d0917d6aab1f0293a063697610&pageNo=1&pageSize=100&isuse=Y&order=viewDay&personal=Y>

#### 가져온 데이터 파싱 json.load()

In [45]:
items = [data['clip_info']
         for data in json.loads(response.body_as_unicode())['body']['result']]
view_counts = [data['view_count'] for data in json.loads(
    response.body_as_unicode())['body']['result']]
ids = [data for data in json.loads(response.body_as_unicode())[
    'body']['result']]

In [36]:
program = items[0]['program']['program_info']['title']

In [11]:
title = items[0]['title']

In [12]:
regdate = "{}.{}.{}".format(items[0]['regdate'][:4], items[0]['regdate'][4:6], items[0]['regdate'][6:8])

In [48]:
view_columns = ["day", "week", "total"]
view_counts[-1]

{'day': 325,
 'week': 347,
 'total': 429,
 'noParseDay': 2020031505000000325,
 'noParseWeek': 2020031505000000347}

In [49]:
for item, view_count, id in zip(items, view_counts, ids):
    program = item['program']['program_info']['title']
    title = item['title']
    regdate = "{}.{}.{}".format(item['regdate'][:4], item['regdate'][4:6], item['regdate'][6:8])
    for view_column in view_columns:
       view = view_count[view_column]
    pid = id['pick_clip_id']
    link = 'http://www.tving.com/pick/player/top/'+pid

In [50]:
programs = []

for data in json.loads(response.body_as_unicode())['body']['result']:
    program = data['clip_info']['program']['program_info']['title'] 
    programs.append(program)
    title = data['clip_info']['title']
    regdate = "{}.{}.{}".format(data['clip_info']['regdate'][:4],\
                                data['clip_info']['regdate'][4:6],\
                                data['clip_info']['regdate'][6:8])
    view_day = data['view_count']['day']
    view_week = data['view_count']['week']
    view_total = data['view_count']['total']
    pid = data['pick_clip_id']
    link = 'http://www.tving.com/pick/player/top/'+pid
#program, title, regdate, view_day, view_week, view_total
len(programs)

100

### items.py 수정

In [52]:
!cat tiving/tiving/items.py

# -*- coding: utf-8 -*-

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class TivingItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass


In [9]:
%%writefile tiving/tiving/items.py
import scrapy

class TivingItem(scrapy.Item):
    program = scrapy.Field()
    title = scrapy.Field()
    regdate = scrapy.Field()
    view_day = scrapy.Field()
    view_week = scrapy.Field()
    view_total = scrapy.Field()
    link = scrapy.Field()

Overwriting tiving/tiving/items.py


### spider.py 생성

In [54]:
%%writefile tiving/tiving/spiders/spider.py
import scrapy
import json

from tiving.items import TivingItem

class Spider(scrapy.Spider):
    
    name = "Tiving"
    allow_domain = ['tving.com']
    start_urls = ['http://www.tving.com/pick/player/top/top']
    
    def parse(self, response):
        apiKey = response.text.split("apiKey=")[1].split("&")[0]
        page, page_size = 1, 100
        url = 'http://api.tving.com/v2/media/pick/clips?\
screenCode=CSSD0100&networkCode=CSND0900&osCode=CSOD0900&teleCode=CSCD0900\
&apiKey={}&pageNo={}&pageSize={}&isuse=Y&order=viewDay&personal=Y'.format(apiKey, page, page_size)
        yield scrapy.Request(url, callback=self.get_content)
        
    def get_content(self, response):
        
        for data in json.loads(response.body_as_unicode())['body']['result']:
            program = data['clip_info']['program']['program_info']['title'] 
            title = data['clip_info']['title']
            regdate = "{}.{}.{}".format(data['clip_info']['regdate'][:4],\
                                data['clip_info']['regdate'][4:6],\
                                data['clip_info']['regdate'][6:8])
            view_day = data['view_count']['day']
            view_week = data['view_count']['week']
            view_total = data['view_count']['total']
            pid = data['pick_clip_id']
            link = 'http://www.tving.com/pick/player/top/' + pid
            
            item = TivingItem()
            item['program'] = program
            item['title'] = title
            item['regdate'] = regdate
            item['view_day'] = view_day
            item['view_week'] = view_week
            item['view_total'] = view_total
            item['link'] = link
            
            yield item

Writing tiving/tiving/spiders/spider.py


### models.py 작성

In [3]:
%%writefile tiving/tiving/models.py

from sqlalchemy import create_engine, Column, Table, ForeignKey, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Integer, Text

from scrapy.utils.project import get_project_settings

DeclarativeBase = declarative_base()

def db_connect():
    """
    Performs database connection using database settings from settings.py.
    Returns sqlalchemy engine instance
    """
    return create_engine(get_project_settings().get("CONNECTION_STRING"))

def create_table(engine):
    DeclarativeBase.metadata.create_all(engine)
    
class TvingClipDB(DeclarativeBase):
    __tablename__ = "tvingclip"

    id = Column(Integer, primary_key=True)
    program = Column('program', Text)
    title = Column('title', Text)
    regdate = Column('regdate', Text)
    view_day = Column('view_day', Text)
    view_week = Column('view_week', Text)
    view_total = Column('view_total', Text)
    link = Column('link', Text)


Overwriting tiving/tiving/models.py


### pipeline.py 수정

In [66]:
! cat tiving/tiving/pipelines.py

# -*- coding: utf-8 -*-

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


class TivingPipeline(object):
    def process_item(self, item, spider):
        return item


In [9]:
%%writefile tiving/tiving/pipelines.py
from sqlalchemy.orm import sessionmaker
from tiving.models import TvingClipDB, db_connect, create_table

class TivingPipeline(object):
    def __init__(self):
        """
        Initializes database connection and sessionmaker.
        Creates deals table.
        """
        engine = db_connect()
        create_table(engine)
        self.Session = sessionmaker(bind=engine)
        
    def process_item(self, item, spider):
        """Save deals in the database.

        This method is called for every item pipeline component.
        """
        session = self.Session()
        tvingclipdb = TvingClipDB()
        tvingclipdb.program = item["program"]
        tvingclipdb.title = item["title"]
        tvingclipdb.regdate = item["regdate"]
        tvingclipdb.view_day = item["view_day"]
        tvingclipdb.view_week = item["view_week"]
        tvingclipdb.view_total = item["view_total"]
        tvingclipdb.link = item["link"]

        try:
            session.add(tvingclipdb)
            session.commit()
        except:
            session.rollback()
            raise
        finally:
            session.close()

        return item

Overwriting tiving/tiving/pipelines.py


### settings.py 수정

In [60]:
! head -n 22 tiving/tiving/settings.py | tail -n 2

# Obey robots.txt rules
ROBOTSTXT_OBEY = True


In [61]:
! sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' tiving/tiving/settings.py

In [62]:
! head -n 22 tiving/tiving/settings.py | tail -n 2

# Obey robots.txt rules
ROBOTSTXT_OBEY = False


In [55]:
# settings.py에서 파이프라인 연결

In [68]:
! echo "ITEM_PIPELINES = {" >> tiving/tiving/settings.py

In [69]:
! echo "    'tiving.pipelines.TivingPipeline': 300," >> tiving/tiving/settings.py

In [70]:
! echo "}" >> tiving/tiving/settings.py

In [71]:
! tail -n 3 tiving/tiving/settings.py

ITEM_PIPELINES = {
    'tiving.pipelines.TivingPipeline': 300,
}


In [ ]:
# MySQL 설정 
#CONNECTION_STRING = "{drivername}://{user}:{passwd}@{host}:{port}/{db_name}?charset=utf8mb4".format(
#     drivername="mysql",
#     user="root",
#     passwd="password",
#     host="**.***.**.***",
#     port="****",
#     db_name="OTT",
#)

### run.sh 작성

In [1]:
%%writefile run.sh
cd tiving
scrapy crawl Tiving -o tiving_04.csv

Overwriting run.sh


In [57]:
import os
os.chmod("run.sh", 0o764)

In [58]:
! ls -al run.sh

-rwxrw-r-- 1 ubuntu ubuntu 43 Mar 15 17:28 run.sh


In [1]:
! ./run.sh

2020-04-02 10:32:17 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: tiving_vod)
2020-04-02 10:32:17 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.6.9 (default, Jan 17 2020, 08:43:56) - [GCC 7.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.15.0-1060-aws-x86_64-with-debian-buster-sid
2020-04-02 10:32:17 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'tiving_vod', 'FEED_FORMAT': 'csv', 'FEED_URI': 'tiving_vod.csv', 'NEWSPIDER_MODULE': 'tiving_vod.spiders', 'SPIDER_MODULES': ['tiving_vod.spiders']}
2020-04-02 10:32:17 [scrapy.extensions.telnet] INFO: Telnet Password: 079106f7cd9513ce
2020-04-02 10:32:17 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstat